In [ ]:
import sys

sys.path.insert(0, "..")

In [ ]:
from src.scrapper import parse_conllu_file
from src.visualization import plot_frequency_of_
from analyses.utils import (
    get_stats,
    build_counts,
    build_dataframes,
    print_top_words_given_tag,
    display_side_by_side,
)

# English dataset

* Name: ESLSpok
* Link to repository [here](https://github.com/UniversalDependencies/UD_English-ESLSpok/tree/master)
* Training data [here](https://github.com/UniversalDependencies/UD_English-ESLSpok/blob/master/en_eslspok-ud-train.conllu)
* Test data [here](https://github.com/UniversalDependencies/UD_English-ESLSpok/blob/master/en_eslspok-ud-test.conllu)

In [ ]:
train_info = parse_conllu_file("../datasets/en_eslspok-ud-train.conllu")
test_info = parse_conllu_file("../datasets/en_eslspok-ud-test.conllu")

In [ ]:
get_stats(train_info)

In [ ]:
get_stats(test_info)

In [ ]:
train_word_counts, train_tag_counts, train_pair_counts = build_counts(train_info)
test_word_counts, test_tag_counts, test_pair_counts = build_counts(test_info)

In [ ]:
plot_frequency_of_("words", train_word_counts, test_word_counts)

In [ ]:
plot_frequency_of_("tags", train_tag_counts, test_tag_counts)

In [ ]:
plot_frequency_of_("word-pair tag", train_pair_counts, test_pair_counts)

In [ ]:
train_df = build_dataframes(train_info)
test_df = build_dataframes(test_info)

tags = train_df.tags.unique()

In [ ]:
train_dfs = [print_top_words_given_tag(train_df, tag) for tag in tags]
display_side_by_side(*train_dfs)

In [ ]:
test_dfs = [print_top_words_given_tag(test_df, tag) for tag in tags]
display_side_by_side(*test_dfs)

# Catalan dataset

* Name: 
* Link to repository [here]()
* Training data [here]()
* Test data [here]()